In [1]:
import pandas as pd

In [3]:
# Test installation of the transformers library
from transformers import pipeline
nlp = pipeline("sentiment-analysis", framework="pt")

# Test the pipeline with a simple input
text = ("Every time Henry dives into the world of transformers, my heart leaps with "
        "excitement—there’s something truly magical about how self-attention layers "
        "dance together to capture the poetry of language! I absolutely love watching "
        "those attention heads weave context across tokens, like tiny explorers "
        "mapping out meaning in every sentence. Each experiment feels like a grand "
        "adventure: tweaking learning rates, sculpting architectures, and fine-tuning "
        "on fresh datasets sends waves of new insights crashing over me. It’s as if "
        "the very act of training these models sparks a conversation between math, "
        "code, and creativity, reminding me how boundless our capacity to learn and "
        "innovate can be. With every epoch that passes, I find myself more in awe of "
        "the elegant simplicity and profound power of transformers—my unwavering "
        "companions on an endless journey of discovery!"
)
outputs = nlp(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


,label,score
0,POSITIVE,0.999473


In [4]:
# Text classification and sentiment analysis seem similar
classifier = pipeline("text-classification", framework="pt")
outputs = classifier(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


,label,score
0,POSITIVE,0.999473


In [5]:
# Named entity recognition
ner_tagger = pipeline("ner", aggregation_strategy="simple", framework="pt")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


,entity_group,score,word,start,end
0,PER,0.998267,Henry,11,16


In [6]:
# Question answering
reader = pipeline("question-answering", framework="pt")
question = "What do I love about transformers?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


,score,start,end,answer
0,0.380045,218,276,watching those attention heads weave context a...


In [7]:
# Summarization
summarizer = pipeline("summarization", framework="pt")
outputs = summarizer(text, max_length=50, min_length=25, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


 Each experiment feels like a grand adventure: tweaking learning rates, sculpting architectures, and fine-tuning on fresh datasets sends waves of new insights crashing over me. With each epoch that passes, I find myself more in awe of the


In [19]:
# Translation
translator = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn",
    tgt_lang="jpn_Jpan",
    max_length=400,
    framework="pt"
    )
outputs = translator(text, clean_up_tokenization_spaces=True)
print(outputs[0]['translation_text'])

Device set to use mps:0


ヘンリーがトランスフォーマーの世界に潜むたびに,私の心は興奮に跳ね上がります. 言語の詩を捉えるために注意の層が一緒に踊る方法には,本当に魔法のようなものがあります!私は,それらの注意の頭がトークンを通して文脈を織りなすのを眺めることをとても好きです. 小さな探検家のように,それぞれの文に意味を映し出します.すべての実験は,偉大な冒険のように感じます.学習速度を調整し,建築を彫り上げ,新しいデータセットに細かく調整することで,新しい洞察の波が私の上に押し寄せます. これらのモデルを訓練する行為自体が,数学,コード,そして創造力との対話を引き起こし,私たちの学習と革新の能力がどれほど無限のかを思い出させるようなものです. 過ぎゆく時代ごとに,私は,無限の発見の旅で私の仲間たちの優雅なシンプルさと深刻な力にますます驚いています.


In [ ]:
# Translation (English to Japanese) using CTranslate2

"""
Fast EN → JA with CTranslate2-fast NLLB-1.3B
-------------------------------------------
• Model weights:  michaelfeil/ct2fast-nllb-200-distilled-1.3B
• Vocab (SPM):    facebook/nllb-200-distilled-1.3B
• Device: CPU (int8_float32) or change to "cuda" + "int8_float16"
"""

from huggingface_hub import snapshot_download
import sentencepiece as spm
import ctranslate2
import pathlib

# ------------------------------------------------------------------
# 1) Download & cache the model & tokenizer
# ------------------------------------------------------------------
MODEL_REPO = "michaelfeil/ct2fast-nllb-200-distilled-1.3B"
model_dir  = snapshot_download(MODEL_REPO)  # ≈1.4 GB weights

# The shared SPM vocab lives in Meta’s official NLLB-200 repo
SPM_REPO   = "facebook/nllb-200-distilled-1.3B"
spm_root   = snapshot_download(
    SPM_REPO, allow_patterns="sentencepiece.bpe.model")
spm_file   = pathlib.Path(spm_root, "sentencepiece.bpe.model")

# Load SentencePiece
sp = spm.SentencePieceProcessor()
sp.load(str(spm_file))

# ------------------------------------------------------------------
# 2) Instantiate the Translator
# ------------------------------------------------------------------
device   = "cpu"  # or "cuda" if you’ve got an NVIDIA GPU
compute  = "int8_float32" if device == "cpu" else "int8_float16"

translator = ctranslate2.Translator(
    model_dir,
    device=device,
    compute_type=compute
)

# ------------------------------------------------------------------
# 3) The translate() helper
# ------------------------------------------------------------------
def translate(
    texts:list[str],
    src: str = "eng_Latn",
    tgt: str = "jpn_Jpan",
    beam: int = 4
) -> list[str]:
    """
    texts: English sentences
    returns: Japanese sentences
    """
    # 1) SPM-tokenise & add [SRC]…["</s>"]
    batches = [
        [src] + sp.encode(t, out_type=str) + ["</s>"]
        for t in texts
    ]

    # 2) Tell CT2 to seed decoding with [TGT]
    tgt_prefix = [[tgt]] * len(batches)

    # 3) Translate, batching by tokens
    results = translator.translate_batch(
        batches,
        batch_type="tokens",
        max_batch_size=2048,
        beam_size=beam,
        target_prefix=tgt_prefix
    )

    # 4) Post-process & detokenise
    outputs = []
    for r in results:
        hyp = r.hypotheses[0]

        # strip out any leftover tags/EOS
        hyp = [tok for tok in hyp if tok not in (tgt, "</s>")]

        # decode pieces → string
        outputs.append(sp.decode_pieces(hyp))

    return outputs

# ------------------------------------------------------------------
# 4) Smoke test
# ------------------------------------------------------------------
if __name__ == "__main__":
    # sample = ["The quick brown fox jumps over the lazy dog."]
    # Test the pipeline with a simple input
    text = (["Every time Henry dives into the world of transformers, my heart leaps with "
            "excitement—there’s something truly magical about how self-attention layers "
            "dance together to capture the poetry of language! I absolutely love watching "
            "those attention heads weave context across tokens, like tiny explorers "
            "mapping out meaning in every sentence. Each experiment feels like a grand "
            "adventure: tweaking learning rates, sculpting architectures, and fine-tuning "
            "on fresh datasets sends waves of new insights crashing over me. It’s as if "
            "the very act of training these models sparks a conversation between math, "
            "code, and creativity, reminding me how boundless our capacity to learn and "
            "innovate can be. With every epoch that passes, I find myself more in awe of "
            "the elegant simplicity and profound power of transformers—my unwavering "
            "companions on an endless journey of discovery!"]
    )
    print(translate(text)[0])
    # → 素早い茶色のキツネが怠け者の犬を飛び越えた。

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 4310.69it/s]


ヘンリーがトランスフォーマーの世界に潜るたびに 私の心は興奮で跳ね上がります! 言語の詩を捉えるために 自己注意の層がどのように一緒に踊っているかについて 本当に魔法のようなものがあります! 私はそれらの注意の頭がトークンを通して文脈を織りなすのを,小さな探検家のように,各文の意味を映し出すのを観るのが大好きです. それぞれの実験は壮大な冒険のように感じます. 学習率を調整し,アーキテクチャを彫り,新鮮なデータセットを細かく調整することで,新しい洞察の波が私の上に押し寄せます. これらのモデルを訓練する行為自体が数学,コード,創造性との間の会話を引き起こし,私たちの学習と革新の能力がどれほど無限であるかを思い出させます. 時代が経つにつれて,私はトランスフォーマーの優雅さと深い力にますます驚きます. 終わりのない発見の旅の私の揺るぎない仲間です!


In [ ]:
import textwrap

# Function to wrap paragraphs
def wrap_paragraphs(text, width=70):
    paras = text.split("\n\n")
    return "\n\n".join(textwrap.fill(p, width=width) for p in paras)

# Text generation
generator = pipeline("text-generation", framework="pt")
response = "Wow, Henry is so passionate about transformers! "
prompt = text[0] + "\n\nResponse from an AI mentor, like Karpathy:\n\n" + response
outputs = generator(prompt, max_length=200, truncation=True, clean_up_tokenization_spaces=True)
wrapped_text = wrap_paragraphs(outputs[0]['generated_text'], width=72)
print(wrapped_text)

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Every time Henry dives into the world of transformers, my heart leaps
with excitement—there’s something truly magical about how self-attention
layers dance together to capture the poetry of language! I absolutely
love watching those attention heads weave context across tokens, like
tiny explorers mapping out meaning in every sentence. Each experiment
feels like a grand adventure: tweaking learning rates, sculpting
architectures, and fine-tuning on fresh datasets sends waves of new
insights crashing over me. It’s as if the very act of training these
models sparks a conversation between math, code, and creativity,
reminding me how boundless our capacity to learn and innovate can be.
With every epoch that passes, I find myself more in awe of the elegant
simplicity and profound power of transformers—my unwavering companions
on an endless journey of discovery!

Response from an AI mentor, like Karpathy:

Wow, Henry is so passionate about transformers!  I love the way he makes
it clear that 